# Assigment 4

In [82]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [83]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

cl = np.loadtxt("data/irr4-cyl2.constraints")
cf = cl[:, 0].astype(np.int64)
c = cl[:, 1:]

In [84]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])


    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]

    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]

    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []

    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges

            # Look up the opposite face
            g = TT[f, ei]

            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue

            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]

            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()

            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()


            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1


    sqrtl = sqrt(llambda)

    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints

    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=complex)

    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]

        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j

        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl

        index += 1

    assert(b.shape[0] == index)


    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.conjugate().T @ A, A.conjugate().T @ b)

    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [85]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [86]:
R = align_field(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [87]:
b1,b2,n = igl.local_basis(v,f)

u = np.zeros(f.shape[0], dtype=np.complex128)
for i, face in enumerate(cf):
    vec = c[i]
    x = np.dot(vec, b1[face])
    y = np.dot(vec, b2[face])
    u[face] = x + 1j * y

I, J, Vals = [], [], []
for fi in range(f.shape[0]):
    for ei in range(3): #x,y,z
        gi = tt[fi, ei]
        if gi != -1 or gi >= fi:
            i0 = f[fi, ei]
            i1 = f[fi, (ei + 1) % 3]
            edge  = v[i1] - v[i0]
            edge /= np.linalg.norm(edge)

            e_f = np.dot(edge, b1[fi]) + 1j * np.dot(edge, b2[fi])
            e_g = np.dot(edge, b1[gi]) + 1j * np.dot(edge, b2[gi])
            e_f_inv = np.conj(e_f)
            e_g_inv = np.conj(e_g)

            Q_ff =  e_f_inv * np.conj(e_f_inv)
            Q_gg =  e_g_inv * np.conj(e_g_inv)
            Q_fg = -e_f_inv * np.conj(e_g_inv)

            I += [fi, gi, fi, gi]
            J += [fi, gi, gi, fi]
            Vals += [Q_ff, Q_gg, Q_fg, np.conj(Q_fg)]
            # print(Vals)
Q = sp.coo_matrix((Vals, (I, J)), shape=(f.shape[0], f.shape[0])).tocsc()

# linear system under constraints
free_indices = np.setdiff1d(np.arange(f.shape[0]), cf)

Q_ff = Q[free_indices][:, free_indices]
Q_fc = Q[free_indices][:, cf]
b = -Q_fc @ u[cf] # rhs contraints

u[free_indices] = sp.linalg.spsolve(Q_ff, b)

# convert back to 3D vectors
u_3d = np.zeros((f.shape[0], 3))
u_3d += u.real[:, np.newaxis] * b1
u_3d += u.imag[:, np.newaxis] * b2
print("Interpolated field (F x 3):")
print(u_3d)





Interpolated field (F x 3):
[[ 0.06262398 -0.26202209  0.12641933]
 [ 0.00733635  0.48344927  0.01315146]
 [ 0.23075977 -0.3549293  -0.09695053]
 ...
 [-0.00586425  0.26995276 -0.00318161]
 [ 0.01876475  0.18028401  0.03277826]
 [ 0.13268427  0.00437402  0.17184795]]


In [88]:
# plot interpolated vector field
face_centroids = v[f].mean(axis=1)
p = mp.plot(v, f, return_plot=True)

# all vectors (red)
p.add_lines(face_centroids, face_centroids + 0.04 * u_3d, shading={"line_color": "red"})
# constraints (green)
p.add_lines(face_centroids[cf], face_centroids[cf] + 0.04 * u_3d[cf], shading={"line_color": "green"})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

2

In [89]:
A = igl.doublearea(v, f) / 2.0
W = sp.diags(np.repeat(A, 3))
G = igl.grad(v,f)

# print(c.shape) 
# print(f.shape[0]) 
 
u_vec = u_3d.T.reshape(-1, 1)

K = G.T @ W @ G
b = G.T @ W @ u_vec

fixed_vertex = 0
K = K.tolil()
K[fixed_vertex, :] = 0
K[fixed_vertex, fixed_vertex] = 1
b[fixed_vertex] = 0

# rank_K = np.linalg.matrix_rank(K)
# print("Rank of K:", rank_K)
# print("Shape of K:", K.shape)

K = K.tocsc()
s = sp.linalg.spsolve(K, b)

g_vec = G@s
g_vec = g_vec.reshape(3, -1).T

face_centroids = v[f].mean(axis=1)
scalar_field = s.real 
scalar_field -= scalar_field.min()
scalar_field /= scalar_field.max()

p2 = mp.plot(v, f, scalar_field, return_plot = True, shading={"colormap": "viridis"})
p2.add_lines(face_centroids, face_centroids + 0.04 * g_vec, shading={"line_color": "red"})

error = np.linalg.norm(g_vec - u_3d, axis = 1)
print(f"Poisson reconstruction error: \n{error}")



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Poisson reconstruction error: 
[0.07428936 0.55710593 0.26421822 0.18498896 0.38624935 0.62914503
 0.19099942 0.39622874 0.145827   0.18750034 1.05916084 0.30358323
 0.22937839 0.39516187 0.41070319 0.31543406 0.43909194 0.07163422
 0.22307399 0.22875455 0.35423872 0.55495204 0.16591079 0.15808525
 0.15100412 0.09208825 0.46774858 0.17695183 0.41696989 0.10434624
 0.41557485 0.65721504 0.35229857 0.2502552  0.18025318 0.21018945
 0.29168138 0.66466124 0.26005716 0.57395898 0.09659167 0.47435734
 0.42672931 0.0833813  0.36360498 0.46625906 0.48617833 0.32746302
 0.37958361 0.34376738 0.38130812 0.32761239 0.27534664 0.36578216
 0.43681495 0.23364164 0.33060357 0.4050701  0.34006461 0.55501312
 0.35746241 0.30458223 0.85794944 0.62550676 0.46399914 0.43353698
 0.3857629  0.18400603 0.37831472 0.20352698 0.14896054 0.28717548
 0.0978971  0.33493359 0.15452904 0.29686078 0.22499976 0.14850503
 0.8816808  0.25582339 0.51280494 0.07526193 0.11865128 0.45317937
 0.23765856 0.23852578 0.205394